<div style="text-align: right"> &uarr;   Ensure Kernel is set to  &uarr;  </div><br><div style="text-align: right"> 
conda_amazonei_pytorch_latest_p36  </div>

# Custom FastAI Model Inference using PyTorch Base Container

This notebook walks you through setting up FastAI Model you just trained using BYOC for inference using endpoint. First we need to create a Predictor class to accept jpeg images as input and output JSON. The default behaviour is to accept a numpy array.

Update Sagemaker SDK if not already done so

In [1]:
!pip install -U sagemaker

  Using cached sagemaker-2.91.1.tar.gz (534 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.91.1-py2.py3-none-any.whl size=737693 sha256=982244f2b1d5090c72082c1c4f4ae988c22e405b5a8bbf1a05c2734a2c1ee42c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/04/e3/8b/f78ee9433f86f32121824c2e287304bb364f4f600f766da233
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.86.2
    Uninstalling sagemaker-2.86.2:
      Successfully uninstalled sagemaker-2.86.2


Import necessary libraries

In [2]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

Please replace the **your-trained-model-uri** with the S3 URI location of your Fast AI Model from the previous notebook

In [3]:
s3_model_loc='s3://sagemaker-us-east-1-779416346969/script-mode-container-fastai-2022-05-23-18-25-48-446/output/model.tar.gz'

## Update the Image Predictor Class

In the next cell, we will update the predictor class to accept json serializer and deserializer and accept application/x-image content type

In [4]:
from sagemaker import Predictor
jpeg_serializer = sagemaker.serializers.IdentitySerializer("application/x-image")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

### Ineference Script
Create an Inference Script along with any libraries we need installed inside requirements.txt and save them in the inf_src folder

In [5]:
%cat inf_src/serve.py

import logging, requests, os, io, glob, time
from fastai.vision.all import *
from PIL import Image
import json

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

JSON_CONTENT_TYPE = 'application/json'
PNG_CONTENT_TYPE = 'application/x-image'

# loads the model into memory from disk and returns it
def model_fn(model_dir):
    logger.info('model_fn')
    learn = load_learner(os.path.join(model_dir, 'model.pth'))
    return learn

# Deserialize the Invoke request body into an object we can perform prediction on
def input_fn(request_body, content_type=PNG_CONTENT_TYPE):
    logger.info('Deserializing the input data.')
    # process an image uploaded to the endpoint
    # if content_type == PNG_CONTENT_TYPE: return open_image(io.BytesIO(request_body))
    if content_type == PNG_CONTENT_TYPE:
        
        # image_data = Image.open(io.BytesIO(request_body))
        image_data=bytes(request_body)
        return(image_data)
    # process a URL submitted to the endpoint
  

In [6]:
%cat inf_src/requirements.txt

fastai==2.4

### Prepare the model
Using the model and ImagePredictor class from above, prepare the model for deployment as an endpoint and provide a serving script that can upack the model

In [7]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.utils import name_from_base

model=PyTorchModel(model_data=s3_model_loc, 
                   name=name_from_base("fastai-custom-cont-mod"),
                   role=role, 
                   framework_version='1.8.0',
                   py_version='py3',
                   entry_point='inf_src/serve.py',
                   source_dir= 'inf_src',
                   predictor_cls=ImagePredictor)

### Deploy the model 

Deploy the model to the end point using ml.m4.xlarge

In [8]:
%%time 
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

----------!CPU times: user 5.94 s, sys: 1.25 s, total: 7.19 s
Wall time: 5min 8s


In [9]:
print(f'Your endpoint name is \n{predictor.endpoint_name}')

Your endpoint name is 
fastai-custom-cont-mod-2022-05-23-18-46-44-308


## Create a Predictor to make inference 

**NOTE** Replace **your-endpoint-name** with your endpoint name in the cell below

In [11]:
from sagemaker.predictor import Predictor
from sagemaker.predictor import json_serializer

#copy the endpoint name and update from sagemaker console --> inference-endpoints
endpoint_name='fastai-custom-cont-mod-2022-05-23-18-46-44-308'

predictor=Predictor(endpoint_name=endpoint_name, 
                    sagemaker_session=sagemaker_session,serializer=jpeg_serializer)

with open('../data/test/Roundabout/R1.png', 'rb') as f:
    img_byte=f.read()
    print(predictor.predict(img_byte))

b'{"class_name": "Roundabout", "confidence": 0.9826133847236633}'


### Using Boto3 Library

In [12]:
import boto3
client=boto3.client('sagemaker-runtime')
im_name="../data/test/Roundabout/R2.png"

response = client.invoke_endpoint(
EndpointName=endpoint_name,
ContentType='application/x-image',
Body=open(im_name, 'rb').read())

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [13]:
import json
json.loads(response['Body'].read().decode("utf-8"))

{'class_name': 'Roundabout', 'confidence': 0.8730788826942444}

### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [14]:
predictor.delete_endpoint()